In [1]:
import pandas as pd
import re,glob

In [20]:
repSamples=u"/Users/gilles/pCloud Drive/FOD/GB/2015-Data/Samples/Verbes3/"
fileSample=u"MGC-10Ko-02-T10000-F3700-Tirage.pkl"
dfSample=pd.read_pickle(repSamples+fileSample)

# Extraction d'un sous-échantillon
- choisir les *nbFormes* les plus fréquentes (indépendamment du tirage)
- former les paradigmes
- regrouper les colonnes en morphomes
- choisir les *nbMorphomes* les plus fréquents
- extraire les paradigmes des lexèmes qui ont au moins une des cases morphomiques remplie
- extraire les paradigmes des lexèmes qui ont au moins une paire de cases morphomiques remplies

=> *nbMorphomes* choisis  
=> sous-échantillon, sous-échantillon avec paires

In [42]:
nbFormes=1000
df=dfSample.sort_values("freq",ascending=False).head(nbFormes)

In [43]:
dfPar=pd.pivot_table(df,values="phono",columns="case",aggfunc=",".join,index="lexeme")

In [44]:
dfPar.count()

case
ai1S      3
ai3P      1
ai3S     43
fi1P      1
fi1S      7
fi2P      4
fi2S      5
fi3P      1
fi3S     12
ii1P      2
ii1S     21
ii2P      3
ii3P     13
ii3S     67
inf     233
is3S      2
pI1P      1
pI2P      5
pI2S      6
pP       19
pc1S      8
pc2P      2
pc3P      2
pc3S     10
pi1P     12
pi1S     65
pi2P     49
pi2S     31
pi3P     23
pi3S    162
ppFP      1
ppFS     20
ppMP      9
ppMS    144
ps2S      1
ps3P      1
ps3S      9
dtype: int64

In [39]:
jouetCases="inf pi3S ppMS pi3P pi1S".split(" ")

print len(dfPar.dropna(thresh=2))
dfPar.dropna(thresh=2)

38


case,inf,pi3S,ppMS,pi3P,pi1S
lexeme,,,,,
aimer,Eme,Em,NaN,NaN,Em
aller,ale,va,ale,vô,vE
appeler,ap6le,apEl,ap6le,NaN,NaN
arriver,arive,ariv,arive,NaN,ariv
arrêter,arEte,arEt,NaN,NaN,NaN
attendre,atâdr,atâ,NaN,NaN,atâ
avoir,avwar,a,y,ô,E
changer,SâZe,NaN,SâZe,NaN,NaN
comprendre,kôprâdr,NaN,kôpri,NaN,kôprâ
